In [1]:
import pandas as pd
from translate import Translator
import google.generativeai as genai
import PIL.Image
from dotenv import load_dotenv
import os

def AutomaticMenuReviewer(location, language, menuLocation, imageLocation, restaurent, verbose=False):
    data = pd.read_csv(menuLocation)

    if language == "Tamil":
        translator = Translator(to_lang='ta')
    elif language == "Hindi":
        translator = Translator(to_lang='hindi')

    dataLst = []
    for _, row in data.iterrows():
        dataLst.append([row['Name'], row['Type'], row['Quantity'], row['Ingredients'], row['Price']])
    if verbose: print(dataLst)

    load_dotenv()
    genai.configure(api_key=os.getenv('API_KEY'))

    img = PIL.Image.open(imageLocation)
    imgModel = genai.GenerativeModel("gemini-pro-vision")
    response = imgModel.generate_content(["What is the name of the dish\nNote:ouput should be in one word format - \"dish\"", img])
    imgDish = response.text
    response = imgModel.generate_content(["what all ingredients are there in this food image, give me the ingredients in list. always give in english language.\nNote:ouput should be in the format - \"[Burger bun, patty (beef, chicken, or veg), lettuce, tomato, onion, cheese, sauce (mayonnaise, ketchup), pickles, ...]\"", img])
    imgIngredients = response.text
    response = imgModel.generate_content(["I have given you a image of a cooked food, give me 1 min of reading content in english Describe the pictured dish in 30 words not more than that. Analyze the cooking style based on visual clues like browning, charring, or visible moisture. Based on these indicators, predict the flavor profile. Mention potential textures and dominant tastes (e.g., smoky, savory, sweet). the response should not be more than 30 words", img])
    imgDiscription = response.text
    if verbose: print(f"{imgDish}\n{imgIngredients}\n{imgDiscription}")

    generation_config = {"temperature": 0.9, "top_p": 1, "top_k": 1, "max_output_tokens": 2048}
    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
    ]
    txtModel = genai.GenerativeModel(model_name="gemini-1.0-pro",
    generation_config=generation_config,
    safety_settings=safety_settings)

    dishes = data.iloc[:, 0]
    if verbose: print(dishes.to_numpy())
    convo = txtModel.start_chat(history=[])
    convo.send_message(f"What is the overall cusine for {dishes.to_numpy} from (american, indian, chineese)\nNote:ouput should be in one word format - \"cusine\"")
    cusine = convo.last.text
    if verbose: print(cusine)
    convo.send_message(f"Based on the cusine give me two more dishes apart from {dishes}\nNote:ouput should be in the format - \"dish1, dish2\"")
    extraDishes = convo.last.text
    if verbose: print(extraDishes)

    convo = txtModel.start_chat(history=[])
    convo.send_message(f"Give me the index of {imgDish} in the list: {dishes}\nNote:ouput should be in one number format - \"[index]\"")
    dishIndex = convo.last.text
    if verbose: print(dishIndex)
    #imgIngredients = "[Pizza base, tomato sauce, cheese, basil]"
    imgIngredients = imgIngredients[2:-1]
    if dishIndex[0] == '[': dishIndex = dishIndex[1:-1]
    curDish = dataLst[int(dishIndex)]
    if verbose: print(curDish)
    curIngredients = curDish[3] + ", " + imgIngredients
    if verbose: print(curIngredients)

    convo = txtModel.start_chat(history=[])
    convo.send_message(f"for the dish: \"{curDish[0]}\"give me the nutrients with respect to the ingredients: \"{curIngredients}\", type: \"{curDish[1]}\" and quantity: \"{curDish[2]}\"\nNote:ouput should be in the format - \"[calories:100, carbohydrates:75g, ...]\"")
    nutrients = convo.last.text
    if verbose: print(nutrients)
    convo.send_message(f"Is {curDish[4]} rupees for the above dish is (cheap, normal, overpriced) in {location}\nNote:ouput should be in one word format - \"pricing\"")
    pricing = convo.last.text
    if verbose: print(pricing)
    convo.send_message(f"Is the above dish met the fssai standards?\nIf yes give the reason, if on give me the discrepency\nNote:ouput should be in the format - \"yes/no-reason\"")
    fssai = convo.last.text
    if verbose: print(fssai)
    convo.send_message(f"write a review for the above dish in 30 words")
    review = convo.last.text
    if verbose: print(review)
    convo.send_message(f"give me a rating for the above dish out of 5\nNote:ouput should be in one float format - \"[rating]\"")
    rating = convo.last.text
    if verbose: print(rating)

    convo = txtModel.start_chat(history=[])
    convo.send_message(f"give me 5 restarents other then {restaurent} that serve {curDish[0]} in {location}\nNote:ouput should be in the format - \"[restaurent1, restaurent2, ...]\"")
    otherRestaurents = convo.last.text
    if verbose: print(otherRestaurents)

    res = [cusine, review, nutrients[1:-1], pricing, fssai, rating[1:-1], extraDishes, imgIngredients, imgDiscription, otherRestaurents[1:-1]]
    if language == "English":
        return res
    rst = []
    for i in range(len(res)):
        if i == 5 or i == 9:
            rst.append(res[i])
            continue
        rst.append(translator.translate(res[i]))
    return rst

C:\Users\farha\AppData\Local\Temp\ipykernel_12864\3323721014.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\farha\anaconda3\envs\mistralenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(AutomaticMenuReviewer("Coimbatore", "English", "menu.csv", "pic.jpeg", "pizza hut", True))

[['Burger', 'Patty', '1 Nos', 'Burger bun, patty (beef, chicken, or veg), lettuce, tomato, onion, cheese, sauce (mayonnaise, ketchup), pickles', 100], ['Sandwich', 'Layered', '1 Nos', 'Bread slices, filling (vegetables, cheese, meat), butter, sauce (mayonnaise, mustard)', 60], ['Pizza', 'Cheesy', '1 medium', 'Pizza dough, pizza sauce, cheese, toppings (pepperoni, vegetables, etc.), herbs (basil, oregano), olive oil', 300], ['French Fries', 'Crispy', '250 grams', 'Potatoes, oil, salt', 80], ['Hot Dogs', 'Snappier', '2 Nos', 'Hotdog buns, sausages (chicken, pork, beef), mustard, ketchup, onions, relish', 120]]
 pizza
 [Pizza crust, tomato sauce, cheese (mozzarella, Parmesan), basil, oregano, garlic, olive oil, salt, pepper]
 This mouthwatering pizza is a perfect balance of flavors and textures. The crispy crust pairs perfectly with the gooey cheese and tangy tomato sauce. The pizza is topped with fresh basil, which adds a touch of sweetness. The pizza is sure to satisfy your cravings and

In [17]:
import pandas as pd
from translate import Translator
import google.generativeai as genai
import PIL.Image
from dotenv import load_dotenv
import os

def AutomaticMenuReviewer(location, language, menuLocation, imageLocation, verbose=False):
    data = pd.read_csv(menuLocation)

    if language == "Tamil":
        translator = Translator(to_lang='ta')
    elif language == "Hindi":
        translator = Translator(to_lang='hindi')

    dataLst = []
    for _, row in data.iterrows():
        dataLst.append([row['Name'], row['Type'], row['Quantity'], row['Ingredients'], row['Price']])
    if verbose: print(dataLst)

    load_dotenv()
    genai.configure(api_key=os.getenv('API_KEY'))

    img = PIL.Image.open(imageLocation)
    imgModel = genai.GenerativeModel("gemini-pro-vision")
    response = imgModel.generate_content(["What is the name of the dish\nNote:ouput should be in one word format - \"dish\"", img])
    imgDish = response.text
    response = imgModel.generate_content(["what all ingredients are there in this food image, give me the ingredients in list. always give in english language.\nNote:ouput should be in the format - \"[Burger bun, patty (beef, chicken, or veg), lettuce, tomato, onion, cheese, sauce (mayonnaise, ketchup), pickles, ...]\"", img])
    imgIngredients = response.text
    response = imgModel.generate_content(["I have given you a image of a cooked food, give me 1 min of reading content in english Describe the pictured dish in 30 words not more than that. Analyze the cooking style based on visual clues like browning, charring, or visible moisture. Based on these indicators, predict the flavor profile. Mention potential textures and dominant tastes (e.g., smoky, savory, sweet). the response should not be more than 30 words", img])
    imgDiscription = response.text
    if verbose: print(f"{imgDish}\n{imgIngredients}\n{imgDiscription}")

    generation_config = {"temperature": 0.9, "top_p": 1, "top_k": 1, "max_output_tokens": 2048}
    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
    ]
    txtModel = genai.GenerativeModel(model_name="gemini-1.0-pro",
    generation_config=generation_config,
    safety_settings=safety_settings)

    dishes = data.iloc[:, 0]
    if verbose: print(dishes.to_numpy())
    convo = txtModel.start_chat(history=[])
    convo.send_message(f"What is the overall cusine for {dishes.to_numpy} from (american, indian, chineese)\nNote:ouput should be in one word format - \"cusine\"")
    cusine = convo.last.text
    if verbose: print(cusine)
    convo.send_message(f"Based on the cusine give me two more dishes apart from {dishes}\nNote:ouput should be in the format - \"dish1, dish2\"")
    extraDishes = convo.last.text
    if verbose: print(extraDishes)

    convo = txtModel.start_chat(history=[])
    convo.send_message(f"Give me the index of {imgDish} in the list: {dishes}\nNote:ouput should be in one number format - \"[index]\"")
    dishIndex = convo.last.text
    if verbose: print(dishIndex)
    #imgIngredients = "[Pizza base, tomato sauce, cheese, basil]"
    imgIngredients = imgIngredients[2:-1]
    if dishIndex[0] == '[': dishIndex = dishIndex[1:-1]
    curDish = dataLst[int(dishIndex)]
    if verbose: print(curDish)
    curIngredients = curDish[3] + ", " + imgIngredients
    if verbose: print(curIngredients)

    convo = txtModel.start_chat(history=[])
    convo.send_message(f"for the dish: \"{curDish[0]}\"give me the nutrients with respect to the ingredients: \"{curIngredients}\", type: \"{curDish[1]}\" and quantity: \"{curDish[2]}\"\nNote:ouput should be in the format - \"[calories:100, carbohydrates:75g, ...]\"")
    nutrients = convo.last.text
    if verbose: print(nutrients)
    convo.send_message(f"Is {curDish[4]} rupees for the above dish is (cheap, normal, overpriced) in {location}\nNote:ouput should be in one word format - \"pricing\"")
    pricing = convo.last.text
    if verbose: print(pricing)
    convo.send_message(f"Is the above dish met the fssai standards?\nIf yes give the reason, if on give me the discrepency\nNote:ouput should be in the format - \"yes/no-reason\"")
    fssai = convo.last.text
    if verbose: print(fssai)
    convo.send_message(f"write a review for the above dish in 30 words")
    review = convo.last.text
    if verbose: print(review)
    convo.send_message(f"give me a rating for the above dish out of 5\nNote:ouput should be in one float format - \"[rating]\"")
    rating = convo.last.text
    if verbose: print(rating)

    res = [cusine, review, nutrients[1:-1], pricing, fssai, rating[1:-1], extraDishes, imgIngredients, imgDiscription]
    if language == "English":
        return res
    rst = []
    for i in range(len(res)):
        if i == 5:
            rst.append(res[i])
            continue
        rst.append(translator.translate(res[i]))
    return rst

In [18]:
print(AutomaticMenuReviewer("Coimbatore", "Tamil", "menu.csv", "pic.jpeg", True))

[['Burger', 'Patty', '1 Nos', 'Burger bun, patty (beef, chicken, or veg), lettuce, tomato, onion, cheese, sauce (mayonnaise, ketchup), pickles', 100], ['Sandwich', 'Layered', '1 Nos', 'Bread slices, filling (vegetables, cheese, meat), butter, sauce (mayonnaise, mustard)', 60], ['Pizza', 'Cheesy', '1 medium', 'Pizza dough, pizza sauce, cheese, toppings (pepperoni, vegetables, etc.), herbs (basil, oregano), olive oil', 300], ['French Fries', 'Crispy', '250 grams', 'Potatoes, oil, salt', 80], ['Hot Dogs', 'Snappier', '2 Nos', 'Hotdog buns, sausages (chicken, pork, beef), mustard, ketchup, onions, relish', 120]]
 Pizza.
 [Pizza crust, tomato sauce, mozzarella cheese, basil leaves]
 This is a cheese pizza. The pizza has a thin crust that is golden brown and crispy. The cheese is melted and bubbly. There are a few basil leaves on top of the pizza. The pizza is cut into slices. The pizza has a slightly smoky flavor from the wood-fired oven. The cheese is salty and gooey. The crust is crispy a